In [1]:
from transformers import pipeline, set_seed
import ipywidgets as widgets
from IPython.display import display, Markdown
import torch

# Optional: fix the seed for reproducibility
set_seed(42)

# Step 1: Dropdown for selecting GPT model
model_selector = widgets.Dropdown(
    options=[
        ('Distilled GPT-2', 'distilgpt2'),
        ('GPT-2 Large', 'gpt2-large'),
        ('GPT-Neo 1.3B', 'EleutherAI/gpt-neo-1.3B'),
        ('GPT-Neo 2.7B', 'EleutherAI/gpt-neo-2.7B'),
        ('GPT-J 6B', 'EleutherAI/gpt-j-6B'),
        ('Falcon-7B Instruct (High RAM)', 'tiiuae/falcon-7b-instruct')
    ],
    description='Select Model:',
    value='EleutherAI/gpt-neo-1.3B',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

# Step 2: Input text box for prompt/topic
user_input = widgets.Text(
    value='The future of AI is',
    placeholder='Type your topic here...',
    description='Input Topic:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Step 3: Generate button
generate_button = widgets.Button(
    description="Generate Text",
    button_style='success'
)

# Step 4: Output area
output_area = widgets.Output()

# Step 5: Generate text function
def generate_text(_):
    output_area.clear_output()
    prompt = user_input.value.strip()
    model_id = model_selector.value
    
    if not prompt:
        with output_area:
            display(Markdown("⚠️ **Please enter a topic first.**"))
        return
    
    with output_area:
        try:
            # Prefer GPU if available
            device = 0 if torch.cuda.is_available() else -1
            print(f"🚀 Loading model `{model_id}` on {'CUDA' if device == 0 else 'CPU'}...")
            
            generator = pipeline("text-generation", model=model_id, device=device)
            result = generator(
                prompt,
                max_length=250,
                do_sample=True,
                temperature=0.8,
                top_k=50,
                top_p=0.95,
                repetition_penalty=1.5,  # Stronger repetition control
                truncation=True,
                pad_token_id=50256
            )[0]["generated_text"]
            
            # Remove incomplete last sentence
            def remove_incomplete_sentence(text):
                # Split by common sentence endings
                sentences = []
                current_sentence = ""
                
                for char in text:
                    current_sentence += char
                    if char in '.!?':
                        sentences.append(current_sentence.strip())
                        current_sentence = ""
                
                # If there's remaining text that doesn't end with punctuation, it's incomplete
                if current_sentence.strip() and not current_sentence.strip()[-1] in '.!?':
                    # Don't add the incomplete sentence
                    pass
                else:
                    # Add the last sentence if it's complete
                    if current_sentence.strip():
                        sentences.append(current_sentence.strip())
                
                return ' '.join(sentences)
            
            # Clean the result
            cleaned_result = remove_incomplete_sentence(result)
            
            # Display the cleaned generated text
            display(Markdown(f"### 📝 Generated Text:\n\n{cleaned_result}"))
            
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                display(Markdown("❌ **CUDA Out of Memory!** Try switching to a smaller model."))
            else:
                display(Markdown(f"⚠️ **Error:** {str(e)}"))

generate_button.on_click(generate_text)

# Step 6: Display all components
display(model_selector, user_input, generate_button, output_area)

2025-06-26 05:58:13.389036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750917493.606398      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750917493.662507      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Dropdown(description='Select Model:', index=2, layout=Layout(width='60%'), options=(('Distilled GPT-2', 'disti…

Text(value='The future of AI is', description='Input Topic:', layout=Layout(width='80%'), placeholder='Type yo…

Button(button_style='success', description='Generate Text', style=ButtonStyle())

Output()